## Erstellung der Korpora im Rahmen des Projekts *Inhaltliche Analyse von Plenarprotokollen des deutschen Bundestages durch Topic Modeling*

In diesen Notebook sind die Codeabschnitte aufgeführt, welche Schritt für Schritt die Korpora erstellen, die für das Projekt verwendet wurden. Das Notebook dient damit der Nachvollziehbarkeit.

Es wird mit dem Originalkorpus gestartet. **Das Originalkorpus sollte sich in einem Ordner auf der selben Dateiebene wie dieses Notebook befinden.** Alle weiteren Ordner werden automatisch erstellt. Es werden nicht nur die Korpora erstellt wie sie auf GitHub zu finden sind - auch zwei kleinere Schritte werden als einzelne Ordner ausgegeben, damit der ganze Workflow anhand der Dateien nachvollzogen werden kann.

Der Code wird nur ausführbar sein, wenn die zu erstellenden Ordner nicht vorhanden sind. Bei erneutem Ausführen des Skriptes sollten die entsprechenden Ordner also wieder entfernt werden.

Hier sind zunächst die Abhängigkeiten des Codes. Die Anforderungen sind:

- cophi==1.0.5

- numpy==1.15.1

- pandas==0.23.4

In [ ]:
import cophi
import glob
import re
import os
import pandas as pd
import numpy as np

### 1. Umwandlung der Dateinamen

Als erstes sollten die Dateinamen an das Sitzungsdatum angepasst werden. Dafür wurde eine kurze Funtkion geschrieben, welche die Monate in Zahlen umwandelt. Hier fiel auf, dass die Dateien unterschiedliche Kodierungen besaßen, weswegen der Umlaut extra behandelt wurde. **Das Originalkorpus auf GitHub wurde bereits vorsorglich nach UTF8 konvertiert, daher sollten Kodierungsschwierigkeiten in diesem Fall eliminiert sein.** Bei diesem Vorgehen handelt es sich um ein Relikt (zumindest für das jetzt konvertierte Korpus), welches das Ergebnis allerdings nicht beeinflusst.

Der Dateinordner des Originalkorpus ist in diesem Fall *Protokolle_original*. Das Sitzungsdatum wird aus den Dateien extrahiert. Der Name kann in der vorletzen Zeile dieses Abschnitts angepasst werden. Es wird ein Ordner mit dem Namen *Protokolle_umbenannt* erstellt. **Das Originalkorpus wird dabei komplett ersetzt - die Originaldateien werden gelöscht.**

In [ ]:
def month_converter(month):
    if month == "März":
        return 3
    months = ['Januar', 'Februar', 'MÃ¤rz', 'April', 'Mai', 'Juni', 'Juli', 'August', 'September', 'Oktober', 'November', 'Dezember']
    return months.index(month) + 1

In [ ]:
def create_new_names(txt_files):
    os.makedirs('Protokolle_umbenannt')
    for file in txt_files:
        
        try:
            f = open(file, encoding = "ISO-8859-1")
            text = f.read()
            # Extraktion des Datums
            date = re.findall("(\d+)\. (\S+) (\d{4})" , text )[0] 
       
            # Gruppe 0 des regulären Ausdrucks ist der Tag 
            day = date[0]
            # " " 1 " " ist der Monat (hier noch im Textformat)
            month = date[1] 
            # " " 2 " " ist das Jahr
            month = month_converter(month)
            year = date[2]
            
            # Zwecks der Sortierung im Dateiensystem werden Nullen angehängt wo nötig
            if int(day) <= 9:
                day = '0'+day
            if month <=9:
                month = '0'+str(month) 
   
        finally:
            if f is not None:
                   f.close()
    
        os.replace(file, 'Protokolle_umbenannt'+ '\\' + str(year) + "_" + str(month) + "_" + str(day) + ".txt")

        
txt_files_original = glob.glob("Protokolle_original/*.txt")        
create_new_names(txt_files_original)

### 2. Entfernen der vorangehenden Tagesordnungspunkte und des Anhangs

Der im vorherigen Schritt erstellte Ordner wird an dieser Stelle wieder eingelesen. Es werden die Tagesordnungspunkte (im Bericht bis zum Stichwort *Beginn* und der Uhrzeit) sowie auch die Anhänge (ab dem Stichwort *Schluss* und der Uhrzeit) entfernt.

Ein neuer Ordner *Protokolle_Zeit* wird erstellt. Dieser entspricht dem auf GitHub.

In [ ]:
txt_files_correctnames = glob.glob("Protokolle_umbenannt/*.txt")

In [ ]:
def remove_pre_and_post(txt_files):
    os.makedirs('Protokolle_Zeit')
    for file in txt_files:
        f = open(file, encoding = "utf-8")
   
        text = f.read()
        # Tagesordnungspunkte löschen
        text4 = re.sub("Deutscher Bundestag(.|\n)+?Beginn:? ?\d+ ?.\d+ ? ?Uhr ?\n", '', text) 
        # Anhänge löschen
        text5 = re.sub("Schluss:? \d+ ?. ?\d+((.|\n)*)", '', text4) 
        file = re.findall('20.+', file)
    
        with open(os.path.join('Protokolle_Zeit', file[0]), 'a', encoding='utf-8') as myfile:
            myfile.write(text5)
            
remove_pre_and_post(txt_files_correctnames)

### 3. Übertragen des Korpus in ein Parteien-Korpus

Mit diesen bereinigten Berichten sollte dann das Parteienkorpus erstellt werden. Erst wurden alle Arten von Zurufen entfernt.
Dann wurden mit einem regulären Ausdruck alle einzelnen Reden identifiziert. Hierbei wurden auch andere Arten von Reden herausgefiltert (nach Bundesland statt nach Partei) sowie auch andere Dateien erstellt, die auf einen durch die vorherige Funktion nicht entdeckten Anhang zurückzuführen sind. **Diese Dateien müssen manuell vor dem nächsten Schritt aus dem Ordner *Protokolle_Parteien* gelöscht werden, sodass nur die Dokumente der Parteien übrig bleiben.** Dieser Ordner ist dann identisch mit dem auf GitHub.

In [ ]:
txt_files_removed = glob.glob("Protokolle_Zeit/*.txt")

In [ ]:
def convert_to_party_corpus(txt_files):
    os.makedirs('Protokolle_Parteien')
    for file in txt_files:
        f = open(file, encoding = "utf-8")
  
        text = f.read()
        # Entfernen von Einwürfen
        text = re.sub("\(.*(Beifall|Zuruf|Heiterkeit|Lachen){1}.*\)", '', text) 
        
        # Regulärer Ausdruck zum identifizieren der Reden
        p = re.compile(r'(^.*\(.+\): ?\n(.|\n)+?)(?=(^ ? ?Präsident.+: ?\n|^ ? ?Vizepräsident.+:|\(.+\): ?\n|^ ? ?.+: ?\n))', re.MULTILINE)
        liste = re.findall(p, text)
        for l in liste:
            l = l[0]
            # für jede Rede die Partei identifizieren
            party = re.findall('\((.+)\):', l)[0] 
            # eventuelle Kodierungsfehler bei BÜNDNIS 90/DIE GRÜNEN beheben
            party2 = re.sub('/|\xa090/| |90|-|.*\(', '', party) 
  
            l = re.sub("\(.*\[.*\)", '', l)
  
            # ein Dokument für die Rede je nach Partei festlegen und die Reden dort nacheinander hineinschreiben
            with open(os.path.join('Protokolle_Parteien', party2+'.txt'), 'a', encoding='utf-8') as myfile: 
                myfile.write(l)
                myfile.write('\n')
                myfile.write('---------')
                   
                    
convert_to_party_corpus(txt_files_removed)

### 4. Entfernen allen Rednernamen aus dem Parteien-Korpus

Um das Parteienkorpus für Topic Modeling vorzubereiten sollten noch die Rednernamen und auch die Partei aus dem Text entfernt werden, damit diese keinen zu großen Einfluss auf die Topics haben können.

Es entsteht ein neuer Ordner *Protokolle_ParteienOhneNamen*. Es wird nur noch der erste Redner mit Partei aufgeführt.

In [ ]:
txt_files_parties = glob.glob("Protokolle_Parteien/*.txt")

In [ ]:
def remove_names(txt_files):
    os.makedirs('Protokolle_ParteienOhneNamen')
    for file in txt_files:
        f = open(file, encoding = "utf-8")
        text = f.read()
        party = re.findall('\((.+)\):', text)[0]   
        party = re.sub('/|\xa090/| |90|-|.*\(', '', party)
        # Identifizieren von Namen und Partei durch die zuvor eingefügte Linie
        p = re.compile('---------.*$', re.MULTILINE) 
        t = re.sub(p, '', text)
    
        with open(os.path.join(r"Protokolle_ParteienOhneNamen", party+'.txt'), 'a', encoding='utf-8') as myfile:
                myfile.write(t)
                
remove_names(txt_files_parties)

### 5. Segmentieren

Im letzten Schritt werden die Partei-Dokumente noch segmentiert, in diesem Fall in Abschnitte von maximal 300000 Wörtern (Tokens). Der Ordner *Protokolle_Parteien_Segmente* wird erstellt, dieser ist wieder identisch mit dem auf GitHub. Satzzeichen und Whitespace wird mit diesem Ansatz zusätzlich entfernt.

In [ ]:
corpus, metadata = cophi.corpus(directory=r"Protokolle_ParteienOhneNamen",
                                filepath_pattern="**/*.txt",
                                treat_as=".txt",
                                encoding="utf-8",
                                lowercase=False,
                                token_pattern=r"\p{L}+\p{P}?\p{L}+",
                                maximum=None)
os.makedirs('Protokolle_Parteien_Segmente')
for doc in corpus.documents:
    # hier wird die Segmentgröße festgelegt
    segments = list(doc.segments(size=300000)) 
    
    # die Partei wird erneut aus dem Text ausgelesen
    party = re.findall('\((.+)\):', doc.text)[0] 
    party = re.sub('/|\xa090/| |90|-|.*\(', '', party)
    party = re.sub('Ü', 'UE', party)
 
    i = 0
    while i < len(segments):
        segment = segments[i]
      
        i = i+1
        if i < 10:
            count = '0'+str(i)
        else: count = str(i)
        # jedes Segment bekommt eine Segmentnummer
        name =  party +'_'+count  

        with open(os.path.join(r"Protokolle_Parteien_Segmente", name+'.txt'), 'a', encoding='utf-8') as myfile:
            for word in segment:
                myfile.write(word + ' ')


### 6. Postprocessing: Zusammenfassen einer Document-Topic Matrix

Für das erstellen der Graphen mit Excel wurde außerdem eine Funktion vorbereitet, welche eine Document-Topic Matrix nach Dokumentnamen zusammenfasst. Dies hat den Zweck bspw. die Matrix nach Monaten oder Jahren zu vereinen. Dafür herangenommen werden die ersten *n* Zeichen des Dokumentnamens. Die Funktion kann zum Beispiel mit einer Dateinamenskonvention wie *2014_01_17.txt* über die ersten 7 Zeichen eine "Tagesmatrix" in eine "Monatsmatrix" umwandeln.

**Die Matrix muss nach dem Dateinamen sortiert sein.** Dies sollte sie diesem Workflow zufolge bereits sein. 

In [ ]:
def denser_matrix(df, characters): 
    i = 0
    n = len(df.index)
    index = df.index.values
    newdf = pd.DataFrame(columns = index)
    newdf = newdf.T
    # Zeilenweise durch das Dataframe iterieren
    while i < n: 
        series = df.iloc[i]
        name = series.name
   
        for index, value in series.iteritems():
            month = str(index)[:characters]
            if month not in newdf.columns:
                # eine leere Zeile mit einem neuen Index im neuen DataFrame anlegen
                newdf[month] = np.nan  
        # Reihenweise je Zeile   
        for index, value in series.iteritems(): 
            # entsprechende zugehörige Zelle im neuen DataFrame identifizieren 
            month = str(index)[:characters] 
            
            # wenn das Feld im neuen Dataframe leer ist
            if pd.isnull(newdf.loc[name, month]): 
                # wird eine neue, leere Liste angelegt
                montharray = []  
                # und der erste Wert wird vorsorglich in das DataFrame geschrieben
                newdf.loc[name, month] = value 
                # sowie als ersten Wert in die Liste geschrieben
                montharray.append(value)   
            # ansonsten wird der Wert der Liste angehängt
            else:
                montharray.append(value) 
                
            final_value = np.mean(montharray) 
            #schließlich ersetzt der Mittelwert der Liste den alten Wert
            newdf.loc[name, month] = final_value 
       
        i = i+1
    return newdf

Hier wird die Dokument-Topcis Datei eingelesen. Auch diese sollte sich im selben Ordner befinden. Bei dem oben genannten Beispiel muss die die Zahl an Zeichen nicht angepasst werden. Mit der letzten Zeile kann außerdem die neue Matrix in eine CSV Datei abgepeichert werden.

In [ ]:
df = pd.read_csv('document_topics.csv', sep=';', encoding ='utf-8', index_col=0)

month_df = denser_matrix(df, 7)
#month_df.to_csv('monate.csv', sep=';', encoding='utf-8')